In [1]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

warnings.filterwarnings(action = 'ignore')

import re
import ast
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split




In [2]:
twitter = pd.read_json("sarcasm_detection_shared_task_twitter_training.jsonl", lines=True)
twitter

,label,response,context
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,SARCASM,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...
...,...,...,...
4995,NOT_SARCASM,@USER You don't . I have purchased a lot on Am...,[@USER Apologies for the inconvenience you fac...
4996,NOT_SARCASM,@USER #Emotions you say 🤔 never knew that I th...,"[@USER 🤔 idk tho , I think I ’ m #hungry . But..."
4997,NOT_SARCASM,"@USER @USER @USER You are so right ... "" Yes !...","[@USER @USER @USER Peace to you , and two coun..."
4998,NOT_SARCASM,@USER @USER @USER Another lazy delusional vote...,[Bernie Sanders told Elizabeth Warren in priva...


In [19]:
input_labels = [ 1 if x=='SARCASM' else 0 for x in twitter.label ]
print(len(input_labels))

5000


In [20]:
def input_sentiments(response, context):
    inputs = []
    analyzer = SentimentIntensityAnalyzer()
    for sentence,c in zip(response, context):
        vs = analyzer.polarity_scores(sentence)
        vs2 = analyzer.polarity_scores(c[-1])
        inputs.append(vs['compound']-vs2['compound'])
    return inputs


In [21]:
inputs = input_sentiments(twitter.response,twitter.context)

In [22]:
twitter_test = pd.read_json("sarcasm_detection_shared_task_twitter_testing.jsonl", lines=True)
twitter_test

,label,context,response,id
0,NOT_SARCASM,"[Well now that ’ s problematic AF <URL>, @USER...","@USER @USER @USER My 3 year old , that just fi...",twitter_1
1,SARCASM,[Last week the Fake News said that a section o...,@USER @USER How many verifiable lies has he to...,twitter_2
2,SARCASM,[@USER Let ’ s Aplaud Brett When he deserves i...,@USER @USER @USER Maybe Docs just a scrub of a...,twitter_3
3,NOT_SARCASM,[Women generally hate this president . What's ...,@USER @USER is just a cover up for the real ha...,twitter_4
4,NOT_SARCASM,"[Dear media Remoaners , you excitedly sharing ...",@USER @USER @USER The irony being that he even...,twitter_5
...,...,...,...,...
1795,NOT_SARCASM,[I have been a business customer of MWeb @USER...,@USER @USER @USER is definitely the best out t...,twitter_1796
1796,SARCASM,[A woman refuses to have her temperature taken...,@USER @USER Ye let her out run wild and infect...,twitter_1797
1797,SARCASM,[The reason big government wants @USER out is ...,"@USER @USER @USER Thanks for that , I would ha...",twitter_1798
1798,NOT_SARCASM,[Happy #musicmonday and #thanks for #all your ...,@USER @USER @USER Yes also #found this on #new...,twitter_1799


In [34]:
test_inputs = input_sentiments(twitter_test.response, twitter_test.context)
test_inputs = np.array(test_inputs).reshape(-1, 1)
print(len(test_inputs))

1800


In [33]:
test_labels = [ 1 if x=='SARCASM' else 0 for x in twitter_test.label ]
print(len(test_labels))

1800


In [29]:

from sklearn.utils import shuffle
X_train, y_train = shuffle(np.array(inputs).reshape(-1, 1), input_labels, random_state=1383944)

In [30]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='rbf', C=10) # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(test_inputs)

In [31]:
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(test_labels, y_pred))

Accuracy: 0.5305555555555556


In [36]:
print("Precision:",metrics.precision_score(test_labels, y_pred))
precision =metrics.precision_score(test_labels, y_pred)

# Model Recall: what perce
# ntage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(test_labels, y_pred))
recall = metrics.recall_score(test_labels, y_pred)

f1 = 2 * (precision * recall) / (precision + recall)
print(f1)

from sklearn.metrics import f1_score

print(f1_score(test_labels, y_pred, average='macro'))

Precision: 0.5247524752475248
Recall: 0.6477777777777778
0.5798110392839384
0.524015022473939


In [68]:
#abs
sarcasm = [abs(inputs[i]) for i in range(0,2499)]
not_sarcasm = [abs(inputs[i]) for i in range(2500,4999)]
#min
print(np.median(sarcasm))
print(np.median(not_sarcasm))
results = []
for i in test_inputs:
    if abs(i)<0.4:
        results.append(1)
        continue
    results.append(0)

c=0
for i,j in zip(results, test_labels):
    if i==j:
        c+=1
print(c/len(test_labels))

0.44630000000000003
0.3818
0.47055555555555556
